# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [185]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

def scrape_books(min_rating, max_price):
    url = "http://books.toscrape.com/catalogue/category/books/romance_8/index.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    book_title_tags = soup.find_all('article', class_='product_pod') #collect all books
    
    book_info = []  #dictionary of book information

    for tag in book_title_tags:
        #name of the book and link for the book detail
        title = tag.h3.a['title'].strip()
        link = tag.h3.a['href']

        #fetching url from book detail page
        book_detail_url = f"http://books.toscrape.com/catalogue/{link[9:]}"
        book_response = requests.get(book_detail_url)
        book_soup = BeautifulSoup(book_response.text, 'html.parser')

        #price and availability from the main url
        price = tag.find('p', class_='price_color').text.strip()
        stock_status = tag.find('p', class_='instock availability').text.strip()

        #
        product_info_table = book_soup.find('table', class_='table table-striped')
        upc_code = None  
        rows = product_info_table.find_all('tr')
        for row in rows:
            if 'UPC' in row.th.text:
                upc_code = row.find('td').text.strip()
                break  

        #grenre from the book detail page
        genre_tag = book_soup.find('a', href=lambda x: x and 'category/books/' in x)
        genre = genre_tag.text.strip() if genre_tag else 'Unknown'

        #rating from the main url
        rating_class = tag.find('p', class_='star-rating')['class'][1]
        rating = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}.get(rating_class, 0)

        #rating and price calculation 
        price_numeric = float(price[1:]) 
        if rating >= min_rating and price_numeric <= max_price:
            #add to dictionar
            book_info.append({
                'UPC': upc_code,
                'title': title,
                'price': price,
                'availability': stock_status,
                'genre': genre,
                'rating': rating,
            })

    #converting to dataframe
    df = pd.DataFrame(book_info)
    return df

book_df = scrape_books(min_rating=4, max_price=20)
display(book_df)

,UPC,title,price,availability,genre,rating
0,9e28048cea8d41f7,First and First (Five Boroughs #3),£15.97,In stock,Romance,4
1,ca71e72655bece85,Something More Than This,£16.24,In stock,Romance,4
2,40c4bf520c17de54,A Gentleman's Position (Society of Gentlemen #3),£14.75,In stock,Romance,5


In [188]:
book_df = scrape_books(min_rating=3, max_price=50)
display(book_df)

,UPC,title,price,availability,genre,rating
0,c2e46a2ee3b4a322,Chase Me (Paris Nights #2),£25.27,In stock,Romance,5
1,00bfed9e18bb36f3,Black Dust,£34.53,In stock,Romance,5
2,9e28048cea8d41f7,First and First (Five Boroughs #3),£15.97,In stock,Romance,4
3,ca71e72655bece85,Something More Than This,£16.24,In stock,Romance,4
4,3a11bb962ff45b78,Doing It Over (Most Likely To #1),£35.61,In stock,Romance,3
5,c5cb9f885d8b3621,The Wedding Pact (The O'Malleys #2),£32.61,In stock,Romance,3
6,df7e628bad5782b4,Dirty (Dive Bar #1),£40.83,In stock,Romance,4
7,b783ce08802cd1d1,Off the Hook (Fishing for Trouble #1),£47.67,In stock,Romance,3
8,40c4bf520c17de54,A Gentleman's Position (Society of Gentlemen #3),£14.75,In stock,Romance,5
9,bd087cf6a846db75,"Sit, Stay, Love",£20.90,In stock,Romance,3
